In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn.functional as F

from omegaconf import OmegaConf
from app.utils import get_data

In [ ]:
OmegaConf.register_new_resolver("eval", resolver=eval, replace=True)

In [ ]:
cfg = OmegaConf.load("../src/app/config/config.yaml")

In [ ]:
train_df, val_df = get_data(cfg, mode="fit")

In [ ]:
from app.data import BYUCustomDataset

In [ ]:
cfg.gamma

In [ ]:
cfg.reduce_mode = "nearest-exact"
cfg.reduce_align_corners = None
cfg.roi_size = [96, 192, 192]
cfg.gamma = 2
dataset = BYUCustomDataset(cfg, df=train_df, mode="train")

In [ ]:
DISPLAY_RANGE = [112, 142]
fig, axs = plt.subplots(10, 3, figsize=(15, 45))
for i in range(30):
    data = dataset.get_data_dict(idx=DISPLAY_RANGE[0] + i)
    row = i // 3
    col = i % 3
    ax = axs[row, col]
    targets = torch.stack(torch.where(data["target"] == 1)).T
    ax.imshow(data["input"][int(targets[0][0])], cmap="gray")
    for _, y, x in targets:
        ax.scatter(
            x + 2, y, color="red", s=20, linewidth=2, marker="o", facecolors="none"
        )
        ax.scatter(
            x, y, color="red", s=20, linewidth=2, marker="o", facecolors="none"
        )  # s=10 pour la taille des points
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
DISPLAY_RANGE = [112, 142]
fig, axs = plt.subplots(10, 3, figsize=(15, 45))
for i in range(5):
    data = dataset[DISPLAY_RANGE[0] + i]
    row = i // 3
    col = i % 3
    ax = axs[row, col]
    targets = torch.stack(torch.where(data["target"] == 1)).T
    ax.imshow(data["input"][0][0][int(targets[0][2])], cmap="gray")
    for _, y, x in targets[:, -3:]:
        ax.scatter(
            x + 2, y, color="red", s=20, linewidth=2, marker="o", facecolors="none"
        )
        ax.scatter(
            x, y, color="red", s=20, linewidth=2, marker="o", facecolors="none"
        )  # s=10 pour la taille des points
    ax.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
x_pooled_cords = []
y_pooled_cords = []
z_pooled_cords = []
x_computed_cords = []
y_computed_cords = []
z_computed_cords = []
true_cords = []
ids = []

for i in range(len(dataset)):
    data = dataset.get_data_dict(idx=i)
    if data["zyx"][0][0] == -1:
        continue
    dim = data["dim"]
    s = data["input"].shape
    zyx = torch.from_numpy(
        train_df.sort_values(by=["z", "y", "x"])
        .loc[train_df["id"] == data["id"], ["z", "y", "x"]]
        .values
    )
    true_cords.append(zyx[0])
    mask = torch.zeros(dim)
    mask[
        zyx[:, 0].to(torch.int),
        zyx[:, 1].to(torch.int),
        zyx[:, 2].to(torch.int),
    ] = 1.0
    pooled_cords = torch.where(
        F.adaptive_max_pool3d(mask.unsqueeze(0).to(torch.float), (s[0], s[1], s[2]))
        == 1.0
    )
    computed_cords = torch.where(data["target"] == 1.0)
    z_pooled_cords.append(pooled_cords[1][0])
    y_pooled_cords.append(pooled_cords[2][0])
    x_pooled_cords.append(pooled_cords[3][0])
    z_computed_cords.append(computed_cords[0][0])
    y_computed_cords.append(computed_cords[1][0])
    x_computed_cords.append(computed_cords[2][0])
    ids.append(data["id"])

In [ ]:
zyxs = torch.cat([t.reshape(1, -1) for t in true_cords], dim=0)

In [ ]:
delta_analysis_df = pd.DataFrame(
    dict(
        z=zyxs[:, 0],
        y=zyxs[:, 1],
        x=zyxs[:, 2],
        z_pooled=torch.tensor(z_pooled_cords).numpy(),
        y_pooled=torch.tensor(y_pooled_cords).numpy(),
        x_pooled=torch.tensor(x_pooled_cords).numpy(),
        z_computed=torch.tensor(z_computed_cords).numpy(),
        y_computed=torch.tensor(y_computed_cords).numpy(),
        x_computed=torch.tensor(x_computed_cords).numpy(),
        id=ids,
    )
)

In [ ]:
scales = []
ids = []

for i in range(len(dataset)):
    data = dataset.get_data_dict(idx=i)
    if data["zyx"][0][0] == -1:
        continue
    scales.append(data["scale"])
    ids.append(data["id"])

In [ ]:
scales = torch.cat([s.reshape(1, -1) for s in scales], dim=0)

In [ ]:
delta_analysis_df = delta_analysis_df.merge(
    pd.DataFrame(
        dict(z_scale=scales[:, 0], y_scale=scales[:, 1], x_scale=scales[:, 2], id=ids)
    ),
    how="inner",
    on="id",
)

In [ ]:
# delta_analysis_df.to_csv("/kaggle/working/delta_analysis.csv", index=False)

In [ ]:
delta_analysis_df["z_pool_err"] = delta_analysis_df["z"] - (
    delta_analysis_df["z_pooled"] / delta_analysis_df["z_scale"]
)
delta_analysis_df["z_comp_err"] = delta_analysis_df["z"] - (
    delta_analysis_df["z_computed"] / delta_analysis_df["z_scale"]
)
delta_analysis_df["y_pool_err"] = delta_analysis_df["y"] - (
    delta_analysis_df["y_pooled"] / delta_analysis_df["y_scale"]
)
delta_analysis_df["y_comp_err"] = delta_analysis_df["y"] - (
    delta_analysis_df["y_computed"] / delta_analysis_df["y_scale"]
)
delta_analysis_df["x_pool_err"] = delta_analysis_df["x"] - (
    delta_analysis_df["x_pooled"] / delta_analysis_df["x_scale"]
)
delta_analysis_df["x_comp_err"] = delta_analysis_df["x"] - (
    delta_analysis_df["x_computed"] / delta_analysis_df["x_scale"]
)

In [ ]:
delta_analysis_df.describe()

In [ ]:
delta_analysis_df["z_scale"].unique()

In [ ]:
train_df.axis0.unique()

In [ ]:
delta_analysis_df["y_scale"].unique()

In [ ]:
plt.scatter(
    delta_analysis_df["y_scale"]
    + delta_analysis_df["z_scale"]
    + delta_analysis_df["x_scale"],
    delta_analysis_df["y_pool_err"],
)

In [ ]:
train_df["volumes"] = train_df.axis0 * train_df.axis1 * train_df.axis2

In [ ]:
delta_analysis_df = delta_analysis_df.merge(
    train_df[["id", "volumes"]], how="inner", on="id"
)